# ILP Formulation - 1

Datasets are all from <br>
http://steinlib.zib.de/steinlib.php

Implementation of "An Integer Programming Formulation of the Steiner Problem in Graphs" 

## Model Formulation

### Sets and Indices
$i, j \in \text{V} = \{1,2, ..,n \}$: Indices and set of vertices. The depot index number is 1.

$\text{E}= \{(i,j)\}$ : Set of edges. The size of $\text{E}$ is $m$

$\text{dE}= \{(i,j)\}$ : Set of edges with direction. The size of $\text{E}$ is $2m$. $(i, j) \neq (j, i)$ 

$\text{S} \subset \text{V}$: The set of terminals(a subset of V). The size of $\text{S}$ is $p$

$\text{G} = (\text{V} , \text{E} , \text{S})$: A graph where the set $\text{V}$ defines the set of vertices, the set $\text{E}$ defines the set of edges and the set $\text{S}$ defines the set of terminals. 

$\text{dG} = (\text{V} , \text{dE} , \text{S})$: A directed graph where the set $\text{V}$ defines the set of vertices, the set $\text{dE}$ defines the set of directed edges and the set $\text{S}$ defines the set of terminals. 

$\text{T} \subset \text{dE}$ : Set of arcs that represents a tree spanning $\text{S}$ in $\text{G}$.

### Parameters 

$c_{ij} \in \mathbb{R}^+$: The cost of an edge or arc $(i, j)$, for all $(i, j) \in \text{dE}$. 

Notice that the cost from vertex $i$ to vertex $j$ is the same as the cost from vertex $j$ to vertex $i$, i.e. $c_{i, j} = c_{j, i}$.

$v_0$ : A random vertex in $\text{S}$ as the source vertex of the tree represented by $\text{T}$

### Decision Variables
$x_{ij} \in \{0, 1\}$: This variable is equal to 1, if arc ij is in the set $\text{T}$. Otherwise, the decision variable is equal to zero.

$u_j \in \{ -1,0,1, ..., n-1\}$: This variable is equal to -1 if $j$ is not included in $\text{T}$. Otherwise, it is equal to the number of arcs in the unique directed $v_0 - j$ path

### Objective Function
- **Minimal Steiner Tree**. Minimize the total cost of $\text{T}$. 

\begin{equation}
\text{Min} \sum_{(i,j) \in \text{dE}}c_{ij} \cdot x_{ij}
\tag{1}
\end{equation}

### Constraints 
- **Single Entrance Constraints**. For each vertex $j$ in set $\text{V}$ except the starting vertex $v_0$, at most 1 arc is included in $\text{T}$ of all the entering arcs $ij$.

\begin{equation}
\sum_{i \in \text{V}-\{j\}} x_{ij} \leq 1  \quad \forall  j \in \text{V}-\{v_0\}
\tag{2}
\end{equation}

- **Range of $u_j$ Constraints Part 1**. For each vertex $j$ in set $\text{V}$ except the starting vertex $v_0$, ensure that the depth of the $j$ in current path is smaller or equal the total number of vertices.(If $j$ is not in set $\text{V}$, both side of the equation will be equal to 0). This ensures that $u_j \leq n - 1$ or $u_j = -1$

\begin{equation}
n \sum_{i \in \text{V}-\{j\}} x_{ij} \geq u_j + 1  \quad \forall  j \in \text{V}-\{v_0\}
\tag{3}
\end{equation}

- **Range of $u_j$ Constraints Part 2**. $u_j$ will be restricted to be real ones with $ u_j \geq 1$

\begin{equation}
(n + 1) \sum_{i \in \text{V}-\{j\}} x_{ij} \leq n(u_j + 1)  \quad \forall  j \in \text{V}-\{v_0\}
\tag{4}
\end{equation}

- **Range of $u_j$ Constraints Part 3**. These constraints ensure that $u_j \geq 0$ if $j$ is one of the terminal vertices

\begin{equation}
u_{v_0} = 0, \quad u_i \geq 0 \quad \forall i \in \text{S}-\{v_0\}
\tag{5}
\end{equation}

- **Change of $u_i$ to $u_j$ of arc $ij$**. The change on $u$ from starting point of an arc to the endpoint. $u_i = u_j - 1$ of arc $ij$
\begin{equation}
1 - n(1-x_{ij}) \leq u_j - u_i \leq 1 + n(1-x_{ij}) \quad \forall ij \in \text{dE}
\tag{6}
\end{equation}

- **Range of $x_{ij}$**. This variable is equal to 1, if arc ij is in the set  T . Otherwise, the decision variable is equal to zero.

\begin{equation}
x_{ij} \in {0, 1} \quad \forall  (i, j) \in \text{dE}
\tag{7}
\end{equation}

### Reading Input Data & Preprocessing
Reading the undirected graph G=(V,E,Z) and trans it into the format dG=(V,dE,Z) 

In [1]:
# Read and save dataframe
import pandas as pd 
def read_df(file_name):
    df = pd.read_csv("../df/"+file_name+".csv")
    return df

def save_df(df, file_name):
    df.to_csv("../df/"+file_name+".csv", index=False)

In [2]:
import numpy as np
def read_graph(name):
    with open(name) as f:
        lines = f.readlines()
        arcs = []
        for line in lines:
            if line == '\n': 
                continue
            parts = line.split()
            det = parts[0]
            if det == 'Name':
                name = parts[1]
            elif det == 'Nodes':
                n_vertices = int(parts[1])
            elif det == 'Edges':
                n_edges = int(parts[1])
            elif det == 'E':
                i = int(parts[1])
                j = int(parts[2])
                c = int(parts[3])
                arcij = ((i,j),c)
                arcji = ((j,i),c)
                arcs.append(arcij)
                arcs.append(arcji)
            elif det == 'Terminals':
                n_terminals = int(parts[1])
        vertices = np.arange(1, int(n_vertices)+1)
        vertices = vertices.tolist()
        terminals = np.arange(1, int(n_terminals)+1)
        terminals = terminals.tolist()
        assert(int(n_edges) == len(arcs)/2)
    f.close()
    ### The format of graphs is dG=(V,dE,Z) 
    return [vertices, arcs, terminals]

In [3]:
# import os
# graphs = []
# path = "../ds/"
# sizes = ["I080/","I160/","I320/"]
# for size in sizes:
#     files = os.listdir(path+size)
#     for file in files:
#         graph = read_graph(path+size+file)
#         graphs.append(graph)

### Due to the limited time, I only used the smallest 100 datasets, the left 200 might take months for calculating

In [5]:
import os
graphs = {}
path = "../ds/"
size = "I080/"
files = os.listdir(path+size)
for file in files:
    file_name = file[:-4]
    graph = read_graph(path+size+file)
    graphs[file_name] = graph

In [6]:
print("The number of datasets used for this project is:{}".format(len(graphs)))

The number of datasets used for this project is:100


### Calculate the optimal solution
With a given graph dG=(V,dE,Z), return the optimal solution in format(optimal edges, optimal vertices, optimal cost)

In [7]:
import gurobipy as gp
from gurobipy import GRB
def formulation_1(graph):
    # read dG=(V,dE,Z) 
    vertices, arcs, terminals = graph
    
    # obtain the size of each set
    n = len(vertices) # n = number of vertices
    m = len(arcs)/2 # m = number of edges
    p = len(terminals) # p = number of ternimals
    
    # choose 1 out of ternimals as the source vertex
    v0 = terminals[0] 

    # delete all the arcs that enter the source vertex
    arcs = [arc for arc in arcs if not arc[0][1] == v0]

    # create the tuple dictionary of arcs
    arcs_dict = gp.tupledict(arcs)
    
    # create the model
    m = gp.Model("Steiner_formulation_1")
    m.Params.PoolSearchMode = 2
    m.Params.PoolGapAbs = 1
    
    # create the decision variables
    # number of variables: 2m + m - 1
    x = m.addVars(arcs_dict.keys(),vtype=GRB.BINARY, name='x')
    u = m.addVars(vertices,lb=-1,ub=n, vtype=GRB.INTEGER, name='u' )
    
    # set up the objective function
    # equation (1)
    m.setObjective(gp.quicksum(arcs_dict[i, j] * x[i, j] for (i, j) in arcs_dict.keys()), GRB.MINIMIZE)
    
    # create the constraints
    vertices = vertices[:v0-1] + vertices[v0:]
    for j in vertices: # number: 3n - 3
        # equation (2)
        m.addConstr(x.sum('*', j) <= 1)
        # equation (3)
        m.addConstr(n * x.sum('*', j) >= u[j] + 1)
        # equation (4)
        m.addConstr((n + 1) * x.sum('*', j) <= n * (u[j] + 1))
        
    for ij in arcs_dict.keys(): # number: 4m
        i = ij[0]
        j = ij[1]
        # equation (6)
        m.addConstr(1 - n * (1 - x[i, j] ) <= u[j] - u[i])
        m.addConstr(1 + n * (1 - x[i, j] ) >= u[j] - u[i])
    
    ternimals = terminals[1:]
    for j in terminals: # number: p - 1
        # equation (7)
        m.addConstr( u[j] >= 0 )
    
    # equation (7)    
    m.addConstr( u[v0] == 0 )
    
    # update the model
    m.update()
    
    # optimize the model
    m.optimize()
    
    # save the optimal solution
    # save the cost
    opt_cost = m.objVal
    
    opt_edges = []
    opt_vertices = []
    
    for sol in range(m.Solcount):
        m.setParam(GRB.Param.SolutionNumber, sol)
        for v in m.getVars():
            # save the edges
            if v.varName.startswith('x') and v.x == 1:
                i,j = v.varName[2:-1].split(',')
                edge = (int(i),int(j))
                opt_edges.append(edge)

            # save the vertices
            if v.varName.startswith('u') and v.x >= 0:
                opt_vertices.append(int(v.varName[2:-1]))
            
    opt_runtime = m.Runtime
    
    return opt_edges, opt_vertices, opt_cost

In [8]:
# # I only calculated the I080 datasets locally, I160 and I320 are calculated in server.
# for file_name in graphs.keys():
#     # compute the optimal solution
#     graph = graphs[file_name]
#     opt_edges, opt_vertices, opt_cost = formulation_1(graph)
#     # To remove duplicates
#     opt_edges = list(set(opt_edges))
#     opt_vertices = list(set(opt_vertices))
    
#     # create a dataframe
#     arcs = graph[1]
#     edges = []
#     for i in range(0, len(arcs), 2):
#         edges.append(arcs[i])
#     df = pd.DataFrame(edges)
#     df[['v1', 'v2']] = pd.DataFrame(df[0].tolist())
#     df['weight'] = pd.DataFrame(df[1].tolist())
#     df['opt_label'] = pd.DataFrame(np.zeros(len(df),dtype=np.int))
#     df = df.drop(columns=[0,1])
#     for (i,j) in opt_edges:
#         df['opt_label'] = np.where((df.v1 == i) & (df.v2 == j) ,1,df.opt_label)
#         df['opt_label'] = np.where((df.v1 == j) & (df.v2 == i) ,1,df.opt_label)
#     # save into csv file
#     save_df(df, file_name)
#     # Now the dataframe is ready for feature engineering

Restricted license - for non-production use only - expires 2022-01-13
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolGapAbs to 1.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 710 rows, 313 columns and 2262 nonzeros
Model fingerprint: 0xbec49d35
Variable types: 0 continuous, 313 integer (233 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [9e+01, 2e+02]
  Bounds range     [1e+00, 8e+01]
  RHS range        [1e+00, 8e+01]
Presolve removed 94 rows and 25 columns
Presolve time: 0.01s
Presolved: 616 rows, 288 columns, 1965 nonzeros
Variable types: 0 continuous, 288 integer (211 binary)

Root relaxation: objective 9.663462e+02, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

     0     0 1551.00000    0   15 1676.00000 1551.00000  7.46%     -    0s
     0     2 1580.00000    0   15 1676.00000 1580.00000  5.73%     -    0s
H   64    20                    1608.0000000 1591.26703  1.04%   5.6    0s
H   74    20                    1607.0000000 1593.73077  0.83%   5.4    0s

Optimal solution found at node 103 - now completing solution pool...

    Nodes    |    Current Node    |      Pool Obj. Bounds     |     Work
             |                    |   Worst                   |
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   103     3          -    9               - 1607.00000      -   4.8    0s

Cutting planes:
  Learned: 2
  Gomory: 1
  Cover: 43
  Implied bound: 39
  Clique: 7
  MIR: 6
  GUB cover: 1
  RLT: 1
  Relax-and-lift: 14

Explored 119 nodes (809 simplex iterations) in 0.13 seconds
Thread count was 16 (of 16 available processors)

Solution count 2: 1607 1608 
No other solutions better than 1e+100

Optimal solution fou

C:\Users\Jiwei\AppData\Local\Temp/ipykernel_10104/1181886147.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['opt_label'] = pd.DataFrame(np.zeros(len(df),dtype=np.int))


 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1196.26923    0    9 1923.00000 1196.26923  37.8%     -    0s
H    0     0                    1911.0000000 1196.26923  37.4%     -    0s
H    0     0                    1713.0000000 1196.26923  30.2%     -    0s
     0     0 1310.38077    0    9 1713.00000 1310.38077  23.5%     -    0s
     0     0 1497.05269    0   15 1713.00000 1497.05269  12.6%     -    0s
     0     0 1497.34356    0   18 1713.00000 1497.34356  12.6%     -    0s
     0     0 1512.08974    0    4 1713.00000 1512.08974  11.7%     -    0s
     0     0 1514.06923    0    6 1713.00000 1514.06923  11.6%     -    0s
     0     0 1521.27500    0    3 1713.00000 1521.27500  11.2%     -    0s
     0     0 1601.46630    0    2 1713.00000 1601.46630  6.51%     -    0s
     0     0 1615.00000    0    2 1713.00000 1615.00000  5.72%     -    0s
     0     0 1616.01282    0    2 1713.00000 1616.01282  5.66%     -    0s
     0     0 1616.0128

     0     0 1513.52627    0   31 3692.00000 1513.52627  59.0%     -    0s
H    0     0                    3206.0000000 1513.52627  52.8%     -    0s
     0     0 1513.52726    0   34 3206.00000 1513.52726  52.8%     -    0s
     0     0 1558.66205    0   47 3206.00000 1558.66205  51.4%     -    0s
     0     0 1559.15198    0   44 3206.00000 1559.15198  51.4%     -    0s
     0     0 1559.15198    0   45 3206.00000 1559.15198  51.4%     -    0s
     0     0 1663.18974    0   29 3206.00000 1663.18974  48.1%     -    0s
H    0     0                    3196.0000000 1663.18974  48.0%     -    0s
     0     0 1669.04252    0   32 3196.00000 1669.04252  47.8%     -    0s
     0     0 1669.22795    0   40 3196.00000 1669.22795  47.8%     -    0s
     0     0 1680.01412    0   37 3196.00000 1680.01412  47.4%     -    0s
     0     0 1680.01875    0   29 3196.00000 1680.01875  47.4%     -    0s
H    0     0                    2789.0000000 1680.01875  39.8%     -    0s
     0     0 1685.00000  

     0     0 1107.09936    0    9          - 1107.09936      -     -    0s
     0     0 1191.12404    0   15          - 1191.12404      -     -    0s
     0     0 1191.13654    0   15          - 1191.13654      -     -    0s
     0     0 1270.02672    0   18          - 1270.02672      -     -    0s
     0     0 1270.05300    0   22          - 1270.05300      -     -    0s
H    0     0                    2692.0000000 1270.05300  52.8%     -    0s
     0     0 1272.01298    0   12 2692.00000 1272.01298  52.7%     -    0s
H    0     0                    2675.0000000 1272.01298  52.4%     -    0s
H    0     0                    2662.0000000 1272.01298  52.2%     -    0s
     0     0 1274.22548    0   11 2662.00000 1274.22548  52.1%     -    0s
H    0     0                    1795.0000000 1274.22548  29.0%     -    0s
     0     0 1279.00001    0   31 1795.00000 1279.00001  28.7%     -    0s
H    0     0                    1658.0000000 1279.00001  22.9%     -    0s
     0     0 1279.18845  


    Nodes    |    Current Node    |      Pool Obj. Bounds     |     Work
             |                    |   Worst                   |
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  8214     0     cutoff   30               - 1381.00000      -   4.9    1s

Cutting planes:
  Learned: 1
  Gomory: 7
  Cover: 135
  Implied bound: 91
  MIR: 1
  Flow cover: 9
  GUB cover: 12
  Inf proof: 4
  Relax-and-lift: 8

Explored 8234 nodes (40593 simplex iterations) in 1.44 seconds
Thread count was 16 (of 16 available processors)

Solution count 1: 1381 
No other solutions better than 1e+100

Optimal solution found (tolerance 1.00e-04)
Best objective 1.381000000000e+03, best bound 1.381000000000e+03, gap 0.0000%
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolGapAbs to 1.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical c

     0     0 1299.72780    0   41 1515.00000 1299.72780  14.2%     -    0s
     0     0 1300.03657    0   42 1515.00000 1300.03657  14.2%     -    0s
     0     0 1300.83513    0   18 1515.00000 1300.83513  14.1%     -    0s
     0     0 1300.94909    0   44 1515.00000 1300.94909  14.1%     -    0s
     0     0 1300.96143    0   45 1515.00000 1300.96143  14.1%     -    0s
     0     0 1303.03846    0    6 1515.00000 1303.03846  14.0%     -    0s
     0     0 1303.03846    0   10 1515.00000 1303.03846  14.0%     -    0s
     0     0 1307.01250    0   25 1515.00000 1307.01250  13.7%     -    0s
     0     0 1307.18736    0   30 1515.00000 1307.18736  13.7%     -    0s
     0     0 1307.19093    0   34 1515.00000 1307.19093  13.7%     -    0s
     0     0 1308.05128    0    8 1515.00000 1308.05128  13.7%     -    0s
     0     0 1308.05987    0   11 1515.00000 1308.05987  13.7%     -    0s
     0     0 1315.79670    0   53 1515.00000 1315.79670  13.1%     -    0s
     0     0 1315.81662  

GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license